In [60]:
import requests
import lxml.html

In [61]:
# 이렇게 하면 javascript 코드가 같이 출력됩니다.
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)

count = 0
for a in root.cssselect('.view_box a'):
    count += 1
    url = a.get('href')
    print(url)
    if count >= 10: break

/store/books/look.php?p_code=B1207366943
javascript:;
/store/books/look.php?p_code=B1207366943
/store/books/look.php?p_code=B6334451644
javascript:;
/store/books/look.php?p_code=B6334451644
/store/books/look.php?p_code=B9882279606
javascript:;
/store/books/look.php?p_code=B9882279606
/store/books/look.php?p_code=B4597837371


In [62]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)

# 모든 링크를 절대 URL로 변환합니다.
root.make_links_absolute(response.url)

# 선택자를 추가해서 명확한 선택을 할 수 있게 합니다.
count = 0
for a in root.cssselect('.view_box .book_tit a'):
    count += 1
    url = a.get('href')
    print(url)
    if count >= 10: break

https://www.hanbit.co.kr/store/books/look.php?p_code=B1207366943
https://www.hanbit.co.kr/store/books/look.php?p_code=B6334451644
https://www.hanbit.co.kr/store/books/look.php?p_code=B9882279606
https://www.hanbit.co.kr/store/books/look.php?p_code=B4597837371
https://www.hanbit.co.kr/store/books/look.php?p_code=B6493543959
https://www.hanbit.co.kr/store/books/look.php?p_code=B2885456605
https://www.hanbit.co.kr/store/books/look.php?p_code=B3346434043
https://www.hanbit.co.kr/store/books/look.php?p_code=B9103889809
https://www.hanbit.co.kr/store/books/look.php?p_code=B1913587019
https://www.hanbit.co.kr/store/books/look.php?p_code=B6446476778


In [63]:
def main():
    """
    크롤러의 메인 처리
    """
    # 여러 페이지에서 크롤링할 것이므로 Session을 사용합니다.
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출합니다.
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    # 제너레이터는 list처럼 사용할 수 있습니다.
    for url in urls:
        print(url)

def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

if __name__ == '__main__':
    main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B1207366943
https://www.hanbit.co.kr/store/books/look.php?p_code=B6334451644
https://www.hanbit.co.kr/store/books/look.php?p_code=B9882279606
https://www.hanbit.co.kr/store/books/look.php?p_code=B4597837371
https://www.hanbit.co.kr/store/books/look.php?p_code=B6493543959
https://www.hanbit.co.kr/store/books/look.php?p_code=B2885456605
https://www.hanbit.co.kr/store/books/look.php?p_code=B3346434043
https://www.hanbit.co.kr/store/books/look.php?p_code=B9103889809
https://www.hanbit.co.kr/store/books/look.php?p_code=B1913587019
https://www.hanbit.co.kr/store/books/look.php?p_code=B6446476778
https://www.hanbit.co.kr/store/books/look.php?p_code=B2831223974
https://www.hanbit.co.kr/store/books/look.php?p_code=B3180984708
https://www.hanbit.co.kr/store/books/look.php?p_code=B2949290774
https://www.hanbit.co.kr/store/books/look.php?p_code=B9483006177
https://www.hanbit.co.kr/store/books/look.php?p_code=B8613567974
https://www.hanbit.co.kr/

추출하고 싶은 요소와 CSS 선택자는 다음과 같습니다.
타이틀 : .store_product_info_box h3  
가격 : .pbr strong  
목차 : #abs_3 .hanbit_edit_view 내부의 p태그들

In [64]:
import requests
import lxml.html

def main():
    # 여러 페이지에서 크롤링할 것이므로 Session을 사용합니다.
    session = requests.Session() 
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출합니다.
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출합니다.
        print(ebook)  # 책 관련 정보를 출력합니다.
        break  # 책 한 권이 제대로 되는지 확인하고 종료합니다.

# book_tit의 a태그에 해당하는 url을 뱉어냄
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출합니다.
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [p.text_content()\
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')]
    }
    return ebook

if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B1207366943', 'title': '구글 엔지니어는 이렇게 일한다', 'price': '40,500', 'content': ['', '\r\n\t\t\t[Part I 전제]\r\n', '\xa0', '\r\n\t\t\tCHAPTER 1 소프트웨어 엔지니어링이란?\r\n', '1.1 시간과 변경', '1.2 규모 확장과 효율성', '1.3 트레이드오프와 비용', '1.4 소프트웨어 엔지니어링 vs 프로그래밍', '1.5 마치며', '1.6 핵심 정리', '\xa0', '\xa0', '\r\n\t\t\t[Part II 문화]\r\n', '\xa0', '\r\n\t\t\tCHAPTER 2 팀워크 이끌어내기\r\n', '2.1 내 코드를 숨기고 싶어요', '2.2 천재 신화', '2.3 숨기는 건 해롭다', '2.4 모든 건 팀에 달렸다', '2.5 마치며', '2.6 핵심 정리', '\xa0', '\r\n\t\t\tCHAPTER 3 지식 공유\r\n', '3.1 배움을 가로막는 장애물', '3.2 철학', '3.3 판 깔아주기: 심리적 안전', '3.4 내 지식 키우기', '3.5 질문 확장하기: 커뮤니티에 묻기', '3.6 지식 확장하기: 누구나 가르칠 게 있다', '3.7 조직의 지식 확장하기', '3.8 가독성 제도: 코드 리뷰를 통한 표준 멘토 제도', '3.9 마치며', '3.10 핵심 정리', '\xa0', '\r\n\t\t\tCHAPTER 4 공정 사회를 위한 엔지니어링\r\n', '4.1 편견은 피할 수 없다', '4.2 다양성이 필요한 이유 이해하기', '4.3 다문화 역량 갖추기', '4.4 다양성 실천하기', '4.5 단일한 접근 방식 거부하기', '4.6 확립된 프로세스에 도전하기', '4.7 가치 vs 결과', '4.8 관심을 잃지 말고 전진하자', '4.9 마치며', '4.10 핵심 정리', '\xa0', '\r\n\t\t\

In [65]:
session = requests.Session()
url = 'https://www.hanbit.co.kr/store/books/look.php?p_code=B1207366943'
response = session.get(url)
root = lxml.html.fromstring(response.content)
print(root.cssselect('.store_product_info_box h3')[0].text_content())
print(root.cssselect('.pbr strong')[0].text_content())

구글 엔지니어는 이렇게 일한다
40,500


## 공백 제거하기

위에서 추출한 정보에는 공백과 의미 없는 문자열이 섞여있습니다. 정규식을 사용해 공백과 빈 문자열을 제거합니다.

In [66]:
import re
import requests
import lxml.html

import requests
import lxml.html

def main():
    # 여러 페이지에서 크롤링할 것이므로 Session을 사용합니다.
    session = requests.Session()  
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출합니다.
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출합니다.
        print(ebook)  # 책 관련 정보를 출력합니다.
        break  # 책 한 권이 제대로 되는지 확인하고 종료합니다.

def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출합니다.
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

def normalize_spaces(s):
    """
    연결돼 있는 공백을 하나의 공백으로 변경합니다.
    """
    return re.sub(r'\s+', ' ', s).strip()

if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B1207366943', 'title': '구글 엔지니어는 이렇게 일한다', 'price': '40,500', 'content': ['[Part I 전제]', 'CHAPTER 1 소프트웨어 엔지니어링이란?', '1.1 시간과 변경', '1.2 규모 확장과 효율성', '1.3 트레이드오프와 비용', '1.4 소프트웨어 엔지니어링 vs 프로그래밍', '1.5 마치며', '1.6 핵심 정리', '[Part II 문화]', 'CHAPTER 2 팀워크 이끌어내기', '2.1 내 코드를 숨기고 싶어요', '2.2 천재 신화', '2.3 숨기는 건 해롭다', '2.4 모든 건 팀에 달렸다', '2.5 마치며', '2.6 핵심 정리', 'CHAPTER 3 지식 공유', '3.1 배움을 가로막는 장애물', '3.2 철학', '3.3 판 깔아주기: 심리적 안전', '3.4 내 지식 키우기', '3.5 질문 확장하기: 커뮤니티에 묻기', '3.6 지식 확장하기: 누구나 가르칠 게 있다', '3.7 조직의 지식 확장하기', '3.8 가독성 제도: 코드 리뷰를 통한 표준 멘토 제도', '3.9 마치며', '3.10 핵심 정리', 'CHAPTER 4 공정 사회를 위한 엔지니어링', '4.1 편견은 피할 수 없다', '4.2 다양성이 필요한 이유 이해하기', '4.3 다문화 역량 갖추기', '4.4 다양성 실천하기', '4.5 단일한 접근 방식 거부하기', '4.6 확립된 프로세스에 도전하기', '4.7 가치 vs 결과', '4.8 관심을 잃지 말고 전진하자', '4.9 마치며', '4.10 핵심 정리', 'CHAPTER 5 팀 이끌기', '5.1 관리자와 테크 리드(혹은 둘 다)', '5.2 개인 기여자에서 리더로', '5.3 엔지니어링 관리자', '5.4 안티패턴', '5.5 올바른 패턴', '5.6 예상 못한 질문', '5.7 그 외 조언과 요령', '5.8 사람은 식물과

## 몽고DB에 저장하기

In [67]:
import time
import re 
import requests
import lxml.html
from pymongo import MongoClient

def scrape_list_page(response):
    """
    목록 페이지의 Response에서 상세 페이지의 URL을 추출합니다.
    """
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출합니다.
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'key': extract_key(response.url),
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': "생략"
    }
    return ebook

def extract_key(url):
    """
    URL에서 키(URL 끝의 p_code)를 추출합니다.
    """
    m = re.search(r"p_code=(.+)", url)
    return m.group(1)

def normalize_spaces(s):
    """
    연결돼 있는 공백을 하나의 공백으로 변경합니다.
    """
    return re.sub(r'\s+', ' ', s).strip()

In [68]:
client = MongoClient('localhost', 27017)
# scraping 데이터베이스의 ebooks 콜렉션
collection = client.scraping.ebooks
# 데이터를 식별할 수 있는 유일키를 저장할 key 필드에 인덱스를 생성합니다.
collection.create_index('key', unique=True)

'key_1'

In [69]:
# 목록 페이지를 추출합니다.
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
# 상세 페이지 URL 목록을 추출합니다.
urls = scrape_list_page(response)

count = 0
for url in urls:
    # URL로 키를 추출합니다.
    key = extract_key(url)
    # MongoDB에서 key에 해당하는 데이터를 검색합니다.
    ebook = collection.find_one({'key': key})
    # MongoDB에 존재하지 않는 경우만 상세 페이지를 크롤링합니다.
    if not ebook:
        time.sleep(1)
        response = requests.get(url)
        ebook = scrape_detail_page(response)
        # 책 정보를 MongoDB에 저장합니다.
        collection.insert_one(ebook)
        count += 1
        if count >= 1: break
    # 책 정보를 출력합니다.
    print(ebook)

In [70]:
ebook

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B1207366943',
 'key': 'B1207366943',
 'title': '구글 엔지니어는 이렇게 일한다',
 'price': '40,500',
 'content': '생략',
 '_id': ObjectId('6273a807a6b4092c15e62a33')}